# 早期准确性测试

包括要完成的核心用力，便于深入了解应用的性能。

一般在早期开发阶段使用。

参考： https://www.trulens.org/trulens_eval/getting_started/quickstarts/groundtruth_evals/

初步结论：

- 如果使用litellm，使用openai，反馈结果为空

In [1]:
%%time

import os

base_url = "http://ape:3000/v1"
api_key = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = base_url

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 12.4 µs


In [2]:
%%time

from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
CPU times: user 6.45 s, sys: 547 ms, total: 6.99 s
Wall time: 6.64 s


In [3]:
%%time

from openai import OpenAI
oai_client = OpenAI(
    base_url = base_url,
    api_key = api_key,
)


from trulens_eval.tru_custom_app import instrument

class APP:
    @instrument
    def completion(self, prompt):
        completion = oai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                temperature=0,
                messages=
                [
                    {"role": "user",
                    "content": 
                    f"Please answer the question: {prompt}"
                    }
                ]
                ).choices[0].message.content
        return completion
    
llm_app = APP()

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 46.9 ms


In [4]:
from trulens_eval import Feedback
from trulens_eval.feedback import GroundTruthAgreement

golden_set = [
    {"query": "who invented the lightbulb?", "response": "Thomas Edison"},
    {"query": "¿quien invento la bombilla?", "response": "Thomas Edison"},
]

f_groundtruth = Feedback(
    GroundTruthAgreement(golden_set).agreement_measure, name="Ground Truth"
).on_input_output()

✅ In Ground Truth, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [5]:
# add trulens as a context manager for llm_app
from trulens_eval import TruCustomApp

tru_app = TruCustomApp(llm_app, app_id="LLM App v1", feedbacks=[f_groundtruth])

In [6]:
# Instrumented query engine can operate as a context manager:
with tru_app as recording:
    llm_app.completion("¿quien invento la bombilla?")
    llm_app.completion("who invented the lightbulb?")

In [7]:
%%time

tru.get_leaderboard(app_ids=[tru_app.app_id])

CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 10.8 ms


,latency,total_cost
app_id,,
LLM App v1,1.5,0.000054
